## Pipeline for producing processed EPC and MCS data and merging them into one table

We're using the asf-core-data repo for the processing.

In [1]:
%load_ext autoreload
%autoreload 2

import os

from asf_core_data.config import base_config

from asf_core_data import generate_and_save_mcs
from asf_core_data import load_preprocessed_epc_data

from asf_core_data.getters.epc import data_batches
from asf_core_data.getters.data_getters import download_core_data, load_s3_data, save_to_s3
from asf_core_data.pipeline.preprocessing import data_cleaning

from asf_core_data.pipeline.preprocessing import preprocess_epc_data
from asf_core_data.pipeline.data_joining import install_date_computation, merge_proc_datasets


2023-03-19 16:47:07,947 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
2023-03-19 16:47:08,718 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/Users/juliasuter/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Users/juliasuter/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Users/juliasuter/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a fut

#### Processing EPC

Currently, we're still handling the EPC processing by downloading and processing it locally. In the future, this will be done directly via S3. 
For now, we need to download the raw EPC data into our local data folder.

In [3]:
LOCAL_DATA_DIR = '/path/to/data/dir'


if not os.path.exists(LOCAL_DATA_DIR):
    os.makedirs(LOCAL_DATA_DIR)

In [ ]:
download_core_data('epc_raw', LOCAL_DATA_DIR, batch='newest')

In [ ]:
# Check whether newest batch shows up a newest in local data dir
print("Local input dir\n---------------")
print("Available batches:", data_batches.get_all_batch_names(data_path=LOCAL_DATA_DIR, check_folder='inputs'))
print("Newest batch:", data_batches.get_most_recent_epc_batch(data_path=LOCAL_DATA_DIR))

In [ ]:
# Process new batch of EPC data
epc_full = preprocess_epc_data.load_and_preprocess_epc_data(
    data_path=LOCAL_DATA_DIR, batch="newest", subset='GB',
    reload_raw=True
)

After processing the EPC data, it has to be uploaded to S3 again for further processing. In the future, this will happen automatically.
In order for the following code to work, you should at least upload the following file to the S3 asf-core-data bucket: `LOCAL_DATA_DIR/BATCH_NAME/EPC_GB_preprocessed.csv`

You can do this using a command as the following in your terminal:

`aws s3 cp LOCAL_DATA_DIR/outputs/EPC/preprocessed_data/2022_Q3_complete/EPC_GB_preprocessed.csv s3://asf-core-data/outputs/EPC/preprocessed_data/2022_Q3_complete/`

#### Processing MCS

Next, we have to process MCS data.

Note that the following two commands might not run correctly in Jupyter notebook because of the COMPANIES_HOUSE_API_KEY. 

If the API KEY is set up correctly according to the instructions [here](https://github.com/nestauk/asf_core_data/pull/37), it will at least run if executed in a terminal: : `python generate_mcs_data.py`

In [ ]:
# Get MCS and join with MCS
uk_geo_data = load_s3_data('asf-core-data', base_config.POSTCODE_TO_COORD_PATH)
generate_and_save_mcs(uk_geo_data, verbose=True)

#### Merging the EPC and MCS

Finally, we merge the EPC and MCS installations and installers data into one dataframe.

The output is a complete datafarame with all EPC records (dedupl) and MCS installations and installers.
We use outer merges to avoid losing data, creating NaN values for missing records.

    - Load EPC data
    - Get best approximation for installation date
    - Merge with MCS installations and reformatting
    - Merge with MCS installers
    - Reformat postcode and geographies
    - Save output to S3

All these steps are summarised in the function `merging_pipeline()` in `merge_proc_datasets.py`. 
Running the full pipeline in a jupyter notebook can lead to a Kernel crash, so executing in a terminal is safer: `python merge_proc_datasets.py`

In [4]:
# Load the processed EPC data 
prep_epc = load_preprocessed_epc_data(data_path="S3", version='preprocessed', batch='newest')

2023-03-19 16:47:24,851 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [5]:
# Add more precise estimations for heat pump installation dates via MCS data
epc_with_MCS_dates = install_date_computation.compute_hp_install_date(
    prep_epc
)

epc_with_MCS_dates.shape

/Users/juliasuter/Documents/repositories/asf_core_data/asf_core_data/pipeline/data_joining/install_date_computation.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["FIRST_HP_MENTION"] = df[identifier].map(dict(first_hp_mention))
/Users/juliasuter/Documents/repositories/asf_core_data/asf_core_data/pipeline/data_joining/install_date_computation.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MCS_AVAILABLE"] = ~df["HP_INSTALL_DATE"].isna()
/Users/juliasuter/Documents/repositories/asf_core_da

(19047896, 64)

The EPC data with enhanced installation dates can then be merged with MCS installation data. This will standardise features such as HP_INSTALLED and HP_TYPE.

In [6]:
epc_mcs_processed = merge_proc_datasets.add_mcs_installations_data(epc_with_MCS_dates, verbose=True)
epc_mcs_processed.columns

EPC (19047896, 65)
MCS (178271, 15)
MCS (EPC matched) (146949, 15)
MCS (EPC unmatched) (31322, 15)
EPC and MCS merged (19086463, 77)


Index(['BUILDING_REFERENCE_NUMBER', 'UPRN', 'ADDRESS1', 'ADDRESS2', 'POSTCODE',
       'INSPECTION_DATE', 'ENERGY_CONSUMPTION_CURRENT', 'TOTAL_FLOOR_AREA',
       'CURRENT_ENERGY_EFFICIENCY', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'CO2_EMISS_CURR_PER_FLOOR_AREA',
       'WALLS_DESCRIPTION', 'WALLS_ENERGY_EFF', 'ROOF_DESCRIPTION',
       'ROOF_ENERGY_EFF', 'FLOOR_DESCRIPTION', 'FLOOR_ENERGY_EFF',
       'WINDOWS_DESCRIPTION', 'WINDOWS_ENERGY_EFF', 'MAINHEAT_DESCRIPTION',
       'MAINHEAT_ENERGY_EFF', 'MAINHEATC_ENERGY_EFF', 'SECONDHEAT_DESCRIPTION',
       'HOTWATER_DESCRIPTION', 'HOT_WATER_ENERGY_EFF', 'LIGHTING_DESCRIPTION',
       'LIGHTING_ENERGY_EFF', 'CO2_EMISSIONS_CURRENT', 'CONSTRUCTION_AGE_BAND',
       'LOW_ENERGY_LIGHTING', 'FLOOR_LEVEL', 'GLAZED_AREA',
       'NUMBER_HABITABLE_ROOMS', 'LOCAL_AUTHORITY_LABEL', 'MAINS_GAS_FLAG',
       'MAIN_HEATING_CONTROLS', 'ENERGY_TARIFF', 'MULTI_GLAZE_PROPORTION',
       'GLAZED_TYPE', 'PHOTO_SUPPLY', 'SOLAR_WATER_HEAT

Now we add the MCS installer data and merge using the installer ID.

In [7]:
# Merge EPC/MCS with MCS installers 
epc_mcs_complete = merge_proc_datasets.add_mcs_installer_data(
    epc_mcs_processed)

epc_mcs_complete.columns

Index(['BUILDING_REFERENCE_NUMBER', 'UPRN', 'ADDRESS1', 'ADDRESS2', 'POSTCODE',
       'INSPECTION_DATE', 'ENERGY_CONSUMPTION_CURRENT', 'TOTAL_FLOOR_AREA',
       'CURRENT_ENERGY_EFFICIENCY', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'CO2_EMISS_CURR_PER_FLOOR_AREA',
       'WALLS_DESCRIPTION', 'WALLS_ENERGY_EFF', 'ROOF_DESCRIPTION',
       'ROOF_ENERGY_EFF', 'FLOOR_DESCRIPTION', 'FLOOR_ENERGY_EFF',
       'WINDOWS_DESCRIPTION', 'WINDOWS_ENERGY_EFF', 'MAINHEAT_DESCRIPTION',
       'MAINHEAT_ENERGY_EFF', 'MAINHEATC_ENERGY_EFF', 'SECONDHEAT_DESCRIPTION',
       'HOTWATER_DESCRIPTION', 'HOT_WATER_ENERGY_EFF', 'LIGHTING_DESCRIPTION',
       'LIGHTING_ENERGY_EFF', 'CO2_EMISSIONS_CURRENT', 'CONSTRUCTION_AGE_BAND',
       'LOW_ENERGY_LIGHTING', 'FLOOR_LEVEL', 'GLAZED_AREA',
       'NUMBER_HABITABLE_ROOMS', 'LOCAL_AUTHORITY_LABEL', 'MAINS_GAS_FLAG',
       'MAIN_HEATING_CONTROLS', 'ENERGY_TARIFF', 'MULTI_GLAZE_PROPORTION',
       'GLAZED_TYPE', 'PHOTO_SUPPLY', 'SOLAR_WATER_HEAT

Finally, we standardise the postcode format and save the output to the S3 bucket.

In [8]:
# Reformat postcode field to include no space
epc_mcs_complete = data_cleaning.reformat_postcode(
    epc_mcs_complete, postcode_var_name="POSTCODE", white_space="remove"
)
epc_mcs_complete['POSTCODE'].head()

0    EH224NH
1    EH222JS
2    EH259RU
3    EH222LS
4    EH222LN
Name: POSTCODE, dtype: object

In [9]:
# Save final merged dataset
save_to_s3(
    base_config.BUCKET_NAME,
    epc_mcs_complete,
    base_config.EPC_MCS_MERGED_OUT_PATH
)

In [10]:
# All in one:
# epc_mcs_combined = merge_proc_datasets.merging_pipeline()